In [60]:
import fastf1
import pandas as pd
import numpy as np

session = fastf1.get_session(2020, 9, 'Race')
session.load()
laps = session.laps


core           INFO 	Loading data for Tuscan Grand Prix - Race [v3.1.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '23', '3', '11', '4', '26', '16', '7', '5', '63', '8', '18', '31', '6', '20', '99', '55', '33', '10']


In [61]:
# get information on the observed car's previous follower
# this follower will most likely be the same as the 'pursuer', but that is not guaranteed
# 
# example. The car we're following is LEC, if the order during lap i-1 is HAM(3rd) NOR(2nd), LEC(1st). 
# Then on lap i-1, Norris pits. Now on lap i LEC only has information on HAM's movement.
# I want to include info about Norris. thus LEC's risk of an undercut
# input: index of the observed car
# outputs: 
#  - prev_follower_pitted (boolean) 1 if the previous follower pitted, 0 else
#  - prev_follower_stint: (int) prev followers stint
#  - prev_follower_pace: (float) prev followers sector two time in seconds
def prev_follower_pitted_funct(ind):
    # gets the observed car's previous lap and their position.
    driver_num = laps.loc[ind, "DriverNumber"]
    lap_num = laps.loc[ind, "LapNumber"]
    
    # get info on previous follower
    if(lap_num == 1 or lap_num == 2 or lap_num == 0):
        # the driver was in 20th place, and thus does not have a follower. Sad. OR
        # this is the second lap, so the prev lap (lap 1) has no real information
        # this is the first lap, and there is no previous lap
        return np.nan, np.nan, np.nan

    orig_driver_prev_pos = laps.pick_lap(lap_number = lap_num - 1).loc[laps['DriverNumber'] == driver_num]["Position"].iloc[0]
    if(orig_driver_prev_pos == 20): 
        return np.nan, np.nan, np.nan
    
    prev_follower = laps.pick_lap(lap_number=lap_num - 1).loc[laps['Position'] == (orig_driver_prev_pos + 1)]

    # gets prevous follower's information    
    prev_follower_pitted = 0
    try:
        if ((not pd.isnull(prev_follower["PitInTime"].iloc[0])) and lap_num != 1.0):
            #driver in front goes in pit (excluding first lap)
            prev_follower_pitted = 1
    except:
        # this is the same positional problem as earlier
        return np.nan, np.nan,np.nan
    prev_follower_stint = int (prev_follower["Stint"].iloc[0])
    prev_follower_pace = prev_follower["LapTime"].iloc[0].total_seconds()
    

    return prev_follower_pitted, prev_follower_stint, prev_follower_pace
    
# returns dictonary of the drivers fastest laps
#     key is the driver number
#     value is time in seconds 
def drivers_fastest_laps(laps):
    drivers_fastest = {}
    for un_d in laps["DriverNumber"].unique():
        fastest_lap_time = laps.pick_drivers(un_d).pick_fastest()["LapTime"]
        if fastest_lap_time == fastest_lap_time:
            # not a nan
            fastest_lap_time = fastest_lap_time.total_seconds()
        drivers_fastest[un_d] = fastest_lap_time
    # laps[0:40].pick_fastest()
    return drivers_fastest

# input: index of lap to get info from
# output: in order:
#    -  in_front_pace:(float) gets the time pursuer took to get to sect2_time in seconds
#    - pursuer_time: (pandas...Timedelta) gets the pursuers_sect2_time
#    - in_front_stint: (int) the stint of the car in front
#    - in_front_pited:(boolean) did the car in front pit?
#    - 
def get_in_front_info(ind):
    pos =  laps.loc[ind, "Position"]
    lap_num = laps.loc[ind, "LapNumber"]
    if pos == 1 or pos == None:
        # this removes the 1st driver from consideration bc they dont have a follower to act on
        return np.nan, np.nan, np.nan, np.nan
    driver_in_front = laps.pick_lap(lap_number=lap_num).loc[laps['Position'] == (pos - 1)]
    if driver_in_front.empty:
        return np.nan, np.nan, np.nan, np.nan
    
    in_front_pitted = 0
    if ((not pd.isnull(driver_in_front["PitInTime"].iloc[0])) and lap_num != 1.0):
        #driver in front goes in pit (excluding first lap)
        in_front_pitted = 1
    if driver_in_front["Stint"].iloc[0] == driver_in_front["Stint"].iloc[0]:
        in_front_stint = int (driver_in_front["Stint"].iloc[0])
    else:
        in_front_stint = np.nan
    in_front_pace = driver_in_front["LapTime"].iloc[0].total_seconds()
    in_front_time = driver_in_front["Sector2SessionTime"].iloc[0]
    return in_front_pace, in_front_time, in_front_stint, in_front_pitted


get_in_front_info(0)

# input: index of driver to get info from
# output: in order:
#    -  pursuer_pace: (float) gets the time pursuer took to get to sect2_time in seconds
#    - pursuer_time:(pandas...Timedelta) gets the pursuers_sect2_time
#    - pursuer_stint: (int) pursuers stint
def get_pursuer_info(ind):
    pos =  laps.loc[ind, "Position"]
    lap_num = laps.loc[ind, "LapNumber"]
    if pos == 20:
        # this removes the 20th driver from consideration (which shouldnt be included in final product)
        return np.nan, np.nan, np.nan
    
    try:
        driver_following = laps.pick_lap(lap_number=lap_num).loc[laps['Position'] == (pos + 1)]
        pursuer_stint = int (driver_following["Stint"].iloc[0])

    except:
        # throws an error when soemone is not in 19th place, but the 20th place person has been knocked out
        # so 19 is last place 
        return np.nan, np.nan, np.nan
    # pursuer_stint = int (driver_following["Stint"].iloc[0])
    pursuer_pace = driver_following["LapTime"].iloc[0].total_seconds()
    pursuer_time = driver_following["Sector2SessionTime"].iloc[0]
    return pursuer_pace, pursuer_time, pursuer_stint

def get_driver_num(ind):
    return laps.loc[ind, "DriverNumber"]



In [62]:
col_names = ['prev_follower_pitted', 'prev_follower_stint', 'prev_follower_pace',
             'in_front_pace', 'in_front_time', 'in_front_stint', 'in_front_pitted',
             'pursuer_pace', 'pursuer_time', 'pursuer_stint',
             ]


laps_pd = pd.DataFrame(laps)

for name in col_names:
    laps_pd[col_names] = np.nan

fastest_laps = drivers_fastest_laps(laps)
fastest_lap_of_race = laps.pick_fastest()["LapTime"].total_seconds()

laps_pd['fastest_laps'] = np.nan
laps_pd['fastest_lap_of_race'] = fastest_lap_of_race

for ind in laps.index:
    print(ind)
    prev_follower_pitted, prev_follower_stint, prev_follower_pace = prev_follower_pitted_funct(ind)
    in_front_pace, in_front_time, in_front_stint, in_front_pitted = get_in_front_info(ind)
    pursuer_pace, pursuer_time, pursuer_stint = get_pursuer_info(ind)
    
    var_names = [ prev_follower_pitted, prev_follower_stint, prev_follower_pace,
             in_front_pace, in_front_time, in_front_stint, in_front_pitted,
             pursuer_pace, pursuer_time, pursuer_stint
             ]
    
    for i in range(len(var_names)):
        laps_pd[col_names[i]].iloc[ind]= var_names[i]
        
    laps_pd['fastest_laps'].iloc[ind] = fastest_laps[get_driver_num(ind)]


/var/folders/5z/md7jx1px78s11b1rm8_cd_ww0000gn/T/ipykernel_10707/2190254296.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_pd[col_names[i]].iloc[ind]= var_names[i]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


/var/folders/5z/md7jx1px78s11b1rm8_cd_ww0000gn/T/ipykernel_10707/2190254296.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_pd['fastest_laps'].iloc[ind] = fastest_laps[get_driver_num(ind)]


81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335

In [63]:
laps_pd

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,prev_follower_pace,in_front_pace,in_front_time,in_front_stint,in_front_pitted,pursuer_pace,pursuer_time,pursuer_stint,fastest_laps,fastest_lap_of_race
0,0 days 00:35:12.350000,GAS,10,NaT,0.0,1.0,NaT,NaT,NaT,NaT,...,NaN,NaN,NaT,1.0,0.0,NaN,NaN,NaN,NaN,78.833
1,0 days 00:35:22.614000,PER,11,0 days 00:01:57.098000,1.0,1.0,0 days 00:00:03.736000,NaT,NaT,0 days 00:00:44.285000,...,NaN,116.335,0 days 00:34:48.448000,1.0,0.0,119.936,0 days 00:34:51.687000,1.0,80.632,78.833
2,0 days 00:37:48.003000,PER,11,0 days 00:02:25.389000,2.0,1.0,NaT,NaT,0 days 00:00:56.326000,0 days 00:00:42.781000,...,NaN,144.786,0 days 00:37:00.474000,1.0,0.0,144.371,0 days 00:37:02.942000,1.0,80.632,78.833
3,0 days 00:40:05.196000,PER,11,0 days 00:02:17.193000,3.0,1.0,NaT,NaT,0 days 00:00:53.834000,0 days 00:00:37.805000,...,144.371,137.351,0 days 00:39:18.554000,1.0,0.0,137.331,0 days 00:39:20.755000,1.0,80.632,78.833
4,0 days 00:42:20.138000,PER,11,0 days 00:02:14.942000,4.0,1.0,NaT,NaT,0 days 00:00:50.119000,0 days 00:00:38.756000,...,137.331,135.385,0 days 00:41:33.398000,1.0,0.0,134.602,0 days 00:41:35.674000,1.0,80.632,78.833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0 days 00:37:59.227000,GIO,99,0 days 00:02:23.099000,2.0,1.0,NaT,NaT,0 days 00:00:52.565000,0 days 00:00:43.067000,...,NaN,142.833,0 days 00:37:10.690000,1.0,0.0,135.826,0 days 00:37:12.471000,1.0,133.809,78.833
782,0 days 00:40:16.033000,GIO,99,0 days 00:02:16.806000,3.0,1.0,NaT,NaT,0 days 00:00:53.307000,0 days 00:00:36.728000,...,135.826,137.245,0 days 00:39:28.124000,1.0,0.0,136.430,0 days 00:39:30.090000,1.0,133.809,78.833
783,0 days 00:42:29.842000,GIO,99,0 days 00:02:13.809000,4.0,1.0,NaT,NaT,0 days 00:00:50.358000,0 days 00:00:37.714000,...,136.430,133.727,0 days 00:41:42.187000,1.0,0.0,134.153,0 days 00:41:44.929000,1.0,133.809,78.833
784,0 days 00:44:44.062000,GIO,99,0 days 00:02:14.220000,5.0,1.0,NaT,NaT,0 days 00:00:53.153000,0 days 00:00:35.400000,...,134.153,134.383,0 days 00:43:56.921000,1.0,0.0,134.103,0 days 00:43:59.468000,1.0,133.809,78.833
